<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Riley/Copy_of_Challenge_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-03 02:52:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2021-09-03 02:52:56 (1.22 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual+Contributions+2013+(UF).csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Individual+Contributions+2013+(UF).csv"), sep=",", header=True)


In [5]:
# Remove multiple columns
list = ['COMMITTEE', 'FILING', 'SCHEDULE', 'PAGENO', 'SEQUENCENO','REFNO', 'STRNO', 'STRNAME', 'APARTMENT', 'OCCUPATION', 'EMPNAME', 'EMPSTRNO', 'EMPSTRNAME',
                'EMPCITY', 'EMPSTATE', 'PREVAMNT', 'REFUNDDATE', 'PAY_METHOD','INTERMNO', 'INTERMNAME', 'INTSTRNO', 'INTSTRNM', 'INTSTRNM', 'INTAPTNO',
                'INTCITY', 'INTST','INTZIP', 'INTEMPNAME', 'INTEMPSTNO', 'INTEMPSTNM', 'INTEMPCITY', 'INTEMPST', 'INTOCCUPA' ,'PURPOSECD', 'EXEMPTCD','ADJTYPECD', 'RR_IND', 'SEG_IND','INT_C_CODE'] 
df = df.drop(*list)
df.show()

+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+----+---------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|     DATE|               NAME|C_CODE|BOROUGHCD|             CITY|STATE|  ZIP|AMNT|MATCHAMNT|
+--------+--------+-------+--------+------------------+---------+-------------------+------+---------+-----------------+-----+-----+----+---------+
|    2013|       5|   1008|       P|Peterson, Daniel P|3/11/2012|   Haywood, David B|   IND|        X|            Bronx|   NY|10458|  20|       20|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|      Herlihy, Neil|   IND|        Z|       Hunnington|   NY|11746|  25|        0|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/5/2012|       Iocca, Karen|   IND|        Z|           Hobken|   NJ| 7030|  25|        0|
|    2013|       5|   1008|       P|Peterson, Daniel P| 3/6/2012|   Lewenberg, Ilana|   IND|        M|         N

In [7]:
df.select('OFFICECD').distinct().show()


+--------+
|OFFICECD|
+--------+
|       3|
|       5|
|       6|
|       1|
|       4|
|       2|
|      IS|
+--------+



In [8]:

df.sample(withReplacement=False,
          fraction = 0.005,
          seed = 101).show()

+--------+--------+-------+--------+------------------+----------+-------------------+------+---------+----------------+-----+-----+----+---------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|      DATE|               NAME|C_CODE|BOROUGHCD|            CITY|STATE|  ZIP|AMNT|MATCHAMNT|
+--------+--------+-------+--------+------------------+----------+-------------------+------+---------+----------------+-----+-----+----+---------+
|    2013|       5|   1008|       P|Peterson, Daniel P| 1/18/2013|    Caliri, Maryann|   IND|        M|        New York|   NY|10016|  50|       50|
|    2013|       5|   1008|       P|Peterson, Daniel P| 5/17/2013|  Matthews, Cynthia|   IND|        M|        New York|   NY|10075|  21|       21|
|    2013|       5|   1008|       P|Peterson, Daniel P| 7/11/2013|   Kelly, Anthony J|   IND|        Q|        Flushing|   NY|11358| 100|      100|
|    2013|       5|   1068|       P|    Ulrich, Eric A| 6/25/2013|Harrygin, Rudranath|   IND|        Q|South Ozo

In [9]:
df2 = df.sample(withReplacement=False,
          fraction = 0.005,
          seed = 101).show()

+--------+--------+-------+--------+------------------+----------+-------------------+------+---------+----------------+-----+-----+----+---------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|         RECIPNAME|      DATE|               NAME|C_CODE|BOROUGHCD|            CITY|STATE|  ZIP|AMNT|MATCHAMNT|
+--------+--------+-------+--------+------------------+----------+-------------------+------+---------+----------------+-----+-----+----+---------+
|    2013|       5|   1008|       P|Peterson, Daniel P| 1/18/2013|    Caliri, Maryann|   IND|        M|        New York|   NY|10016|  50|       50|
|    2013|       5|   1008|       P|Peterson, Daniel P| 5/17/2013|  Matthews, Cynthia|   IND|        M|        New York|   NY|10075|  21|       21|
|    2013|       5|   1008|       P|Peterson, Daniel P| 7/11/2013|   Kelly, Anthony J|   IND|        Q|        Flushing|   NY|11358| 100|      100|
|    2013|       5|   1068|       P|    Ulrich, Eric A| 6/25/2013|Harrygin, Rudranath|   IND|        Q|South Ozo

In [15]:
import pyspark.sql.functions
  
df2 = df2.filter(df2.OFFICECD ==1)
df2.show()


AttributeError: ignored